## Purpose - To scrape data on authors, their affiliation and reported addresses at the level of each paper

### Import Packages

In [27]:
import pandas as pd 
import os 
import numpy as np 
import re
import string
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
import math
from selenium.common.exceptions import NoSuchElementException
import pickle

In [17]:
EXE_PATH = r"C:\Users\nik596\Downloads\chromedriver_win32\chromedriver.exe"
browser = webdriver.Chrome(executable_path=EXE_PATH)

C:\Users\nik596\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  


In [18]:
base = "https://www.nber.org/papers?page=1&perPage=100&sortBy=public_date"
browser.get(base)
browser.implicitly_wait(10)

text = browser.find_element_by_class_name("search__info-results").text
pages = int(text[text.find('F') + 2 : text.find('R') - 1]) + 1
pages = math.ceil(pages/100)
pages

C:\Users\nik596\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:5: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  """


305

In [22]:
#df = pd.read_excel(r'C:\Users\nik596\Dropbox (Harvard University)\New folder\NBER_WP.xlsx')
#scraped = list(df['paper'])
scraped = []

In [24]:
paper_links = []
url = "https://www.nber.org/papers?page={q}&perPage=100&sortBy=public_date"

for i in range(1, pages + 1, 1):
    
    if i%10 ==0:
        print(i)
    
    browser.get(url.format(q = str(i)))
    browser.implicitly_wait(30)
    
    
    for element in browser.find_elements_by_xpath('//a[contains(@href,"/papers/")]'):
        paper_link = element.get_attribute('href')
        paper_links.append(paper_link)

C:\Users\nik596\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:13: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  del sys.path[0]


10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300


In [29]:
paper_links = set(paper_links)
with open(r'C:\Users\nik596\Documents\GitHub\NBER\papers.txt','wb') as f:
   pickle.dump(paper_links, f)

In [ ]:
with open(r'C:\Users\nik596\Documents\GitHub\NBER\papers.txt','rb') as f:
   paper_links = pickle.load(f)

In [ ]:
dic = {}
count = 0

errors = []


for paper_link in paper_links:
        
        try:
            html = urlopen(paper_link)
            soup = BeautifulSoup(html, 'lxml')
        
            paper_name = soup.find('h1', class_ = 'page-header__title').text.strip()
        
            if paper_name not in scraped:
                dic[count] = {'paper': paper_name}
                #print(paper_name)
                
                paper_month = soup.find('time').text
                dic[count]['paper month'] = paper_month
                
                temp = soup.find('div', class_ = 'page-header__citation-item').text.strip()
                dic[count]['paper number'] = temp[temp.find('Paper') + 6:]
                
                for each in soup.find_all('div', class_ = 'info-grid__item'):
                    text = [t.text for t in each.find_all('a')]
                    l = len(text)
    
                    if 'topics' in each.text.lower():
                        for c in range(l):
                            dic[count]['paper topic ' + str(c + 1)] = text[c]
                    elif 'programs' in each.text.lower():
                        for c in range(l):
                            dic[count]['paper program ' + str(c + 1)] = text[c]
                    elif 'working groups' in each.text.lower():
                        for c in range(l):
                            dic[count]['paper group ' + str(c + 1)] = text[c]
                    elif 'projects' in each.text.lower():
                        for c in range(l):
                            dic[count]['paper project ' + str(c + 1)] = text[c]
        
                authors = soup.find_all('div', class_ = 'page-header__authors')[0].find_all('a')
        
                num = 1
                for author in authors:
            
                    author_name = author.text
                    author_link = author.get('href')
                    dic[count]['author ' + str(num)] = author_name
                    dic[count]['author link ' + str(num)] = author_link
            
                    author_link = "https://www.nber.org" + author_link
                    html = urlopen(author_link)
                    soup = BeautifulSoup(html, 'lxml')
            
                    author_position = soup.find_all('div', class_ = 'person-header__title')[0].text.strip()
                    dic[count]['author position ' + str(num)] = author_position
            
                    author_affiliation = soup.find_all('div', class_ = 'person-header__summary')[0].text.strip()
                    dic[count]['author affiliation ' + str(num)] = author_affiliation
                    
                    author_address = soup.find_all('div', 'contact-card__social')[-1].text.strip().replace('\n', ' ')
                    dic[count]['author address ' + str(num)] = author_address
            
                    num+=1
            
                count += 1
                if count%50 ==0:
                    print(count)
                    print(paper_name)
        except:
            errors.append(paper_link)

50
Exchange Rates as Trade Frictions: Estimates and Implications for Policy
100
The Impact of City Contracting Set-Asides on Black Self-Employment and Employment
150
Do Students Know Best?  Choice, Classroom Time, and Academic Performance
200
Measuring Business Cycles Approximate Band-Pass Filters for Economic    Time Series
250
Trading Spaces: Medicare’s Regulatory Spillovers on Treatment Setting for Non-Medicare Patients
300
A Two Covid-19 Years Quartile Comparison of Official with Excess Mortality: Voice and Accountability and the Impact of Vaccines
350
The Purpose of Trade Agreements
400
Earnings Test, Non-actuarial Adjustments and Flexible Retirement
450
Trade Liberalization in Disinflation
500
Determinants of Slave and Crew Mortality in the Atlantic Slave Trade
550
Why Has California's Residential Electricity Consumption Been So Flat since the 1980s?: A Microeconometric Approach
600
Constitutions, Corporations, and Corruption: American States and Constitutional Change
650
Inequal

In [14]:
df=pd.DataFrame.from_dict(dic).T
df.to_excel(r'C:\Users\nik596\Documents\GitHub\NBER\NBER_WP.xlsx',engine='xlsxwriter')

In [15]:
df

,paper,paper month,paper number,paper topic 1,paper topic 2,paper topic 3,paper topic 4,paper topic 5,paper program 1,paper program 2,...,author 26,author link 26,author position 26,author affiliation 26,author address 26,author 27,author link 27,author position 27,author affiliation 27,author address 27
0,The Anatomy of U.S. Sick Leave Schemes: Eviden...,April 2022,29956,"Health, Education, and Welfare",Health,Labor Economics,Labor Supply and Demand,Labor Compensation,Economics of Aging,Economics of Education,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Generalized Separability and Integrability: Co...,April 2022,29997,Microeconomics,Households and Firms,Market Structure and Distribution,Industrial Organization,Market Structure and Firm Performance,Development Economics,Industrial Organization,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Modeling Uncertainty as Ambiguity: a Review,April 2022,29915,Microeconomics,Economics of Information,Macroeconomics,Consumption and Investment,Business Cycles,Asset Pricing,Economic Fluctuations and Growth,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Increasing the Demand for Workers with a Crimi...,April 2022,29947,Econometrics,Experimental Design,Labor Economics,Labor Supply and Demand,Other,Labor Studies,"Productivity, Innovation, and Entrepreneurship",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Welfare Implications of Electric-Bike Subsidie...,April 2022,29913,Public Economics,Taxation,Regional and Urban Economics,NaN,NaN,Environment and Energy Economics,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1328,Mammograms and Mortality: How Has the Evidence...,April 2021,28648,"Health, Education, and Welfare",Health,NaN,NaN,NaN,Health Care,Health Economics,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1329,Medicaid and Fiscal Federalism During the COVI...,April 2021,28670,Subnational Fiscal Issues,COVID-19,NaN,NaN,NaN,Health Care,Health Economics,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1330,Optimal Product Design: Implications for Compe...,April 2021,28638,Microeconomics,Market Structure and Distribution,Macroeconomics,Consumption and Investment,Industrial Organization,Economic Fluctuations and Growth,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1331,Addressing Biases that Impact Homeowners’ Adop...,April 2021,28678,Public Economics,Taxation,Environmental and Resource Economics,Energy,Environment,Environment and Energy Economics,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
